In [1]:
import $ivy.`org.apache.spark::spark-sql:3.3.0`
import $ivy.`org.apache.spark::spark-mllib:3.3.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import $ivy.$                                  

import $ivy.$                                    

import org.apache.log4j.{Level, Logger}


# Spark ML Production

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Spark ML Production")
                .getOrCreate()

import spark.implicits._

spark.sparkContext.setLogLevel("WARN")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


: 

## Загружаем данные

In [3]:
val basePath = "/home/vadim/Projects/otus/SparkML/Credit Card Customers"

basePath: String = "/home/vadim/Projects/otus/SparkML/Credit Card Customers"

In [4]:
val data = spark
        .read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(s"$basePath/data/BankChurners.csv")

data: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 21 more fields]

## Загружаем модель

In [5]:
import org.apache.spark.ml.PipelineModel

val model = PipelineModel.load(s"$basePath/Scala/pipelineModel")

import org.apache.spark.ml.PipelineModel


model: PipelineModel = pipeline_b8e5f2523bdb

## Вычисляем

In [6]:
val predicted = model.transform(data)

predicted: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 37 more fields]

In [7]:
predicted.show(10)

22/07/11 17:10:00 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------+----------------------+-----------------------+---------------------+--------------------+-----------------------------+----------------------------+----------------

## Проверяем результат

In [8]:
val tp = predicted.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 1)).count
val tn = predicted.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 0)).count
val fp = predicted.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 1)).count
val fn = predicted.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 0)).count

println(s"Confusion Matrix:\n$tp\t$fp\n$fn\t\t$tn\n")

Confusion Matrix:
1197	1864
430		6636



tp: Long = 1197L
tn: Long = 6636L
fp: Long = 1864L
fn: Long = 430L

In [9]:
val accuracy = (tp + tn) / (tp + tn + fp + fn).toDouble
val precision = tp / (tp + fp).toDouble
val recall = tp / (tp + fn).toDouble

println(s"Accuracy = $accuracy")
println(s"Precision = $precision")
println(s"Recall = $recall\n")

Accuracy = 0.7734768440801817
Precision = 0.3910486769029729
Recall = 0.7357098955132145



accuracy: Double = 0.7734768440801817
precision: Double = 0.3910486769029729
recall: Double = 0.7357098955132145